# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставимих как есть.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

In [22]:
def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum','mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', 'Доля невозврата']
    display(pivot.sort_values('Доля невозврата', ascending = False))
pivot(data, 'children')# Ваш код будет здесь. Вы можете создавать новые ячейки.

,children,Кол-во клиентов,Кол-во должников,Доля невозврата
4,4,41,4,0.097561
2,2,2052,194,0.094542
1,1,4808,444,0.092346
3,3,330,27,0.081818
0,0,14091,1063,0.075438
5,5,9,0,0.000000


**Вывод:**  Проводя анализ между зависимостью между количеством детей и возвратом кредита в срок можно заметить, что клиенты без детей составляют наименьшую долю невозврата, тут все очевидно, денежных трат меньше. Однако следует обратить внимание на то, что клиенты с 5 детьми имеют 0 долю невозврата,здесь сложно сделать какик-то выводы, т.к. само их количество мало, возможно они только взяли кредит. Далее располагаются клиенты с 3-мя детьми. Если судить по их кол-ву, можно сделать вывод о том, что данные семьи понимают на что они берут кредит и более ответственно подходят к его выплате. После же расположились семьи с 1-им, 2-мя детьми. Самая же высокая доля невозврата у клиентов с 4-мя детьми

In [23]:
def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum','mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', 'Доля невозврата']
    display(pivot.sort_values('Доля невозврата', ascending = False))
pivot(data, 'family_status')# Ваш код будет здесь. Вы можете создавать новые ячейки.

,family_status,Кол-во клиентов,Кол-во должников,Доля невозврата
0,Не женат / не замужем,2796,273,0.097639
3,гражданский брак,4134,385,0.093130
4,женат / замужем,12261,927,0.075606
1,в разводе,1189,84,0.070648
2,вдовец / вдова,951,63,0.066246


**Вывод:** Из данной таблицы нам видно, что у вдовцов/вдов доля невозврата меньше всего. Максимальная же доля невозврата у женатых/замужних пар

In [24]:
def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum','mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', 'Доля невозврата']
    display(pivot.sort_values('Доля невозврата', ascending = False))
pivot(data, 'total_income_category')# Ваш код будет здесь. Вы можете создавать новые ячейки.

,total_income_category,Кол-во клиентов,Кол-во должников,Доля невозврата
4,E,22,2,0.090909
2,C,15921,1353,0.084982
0,A,25,2,0.080000
1,B,5014,354,0.070602
3,D,349,21,0.060172


**Вывод:** Проанализировав данную таблицу можно увидеть, что у клиентов из категории D(доход: от 30.001 до 50.000) доля невозврата составляет наименьший показатель по сравнению с клиентами из категории E(доход: до 30.000). Однако тут сложно делать вывод о наивысшей доле невозврата, т.к. кол-во клиентов у категории E и А(доход: свыше 1.000.000) практически идентичны, так что более правильным будет сюда отнести категорию С(доход: от 50.001 до 200.000)

In [25]:
def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum','mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', 'Доля невозврата']
    display(pivot.sort_values('Доля невозврата', ascending = False))
pivot(data, 'purpose_category')# Ваш код будет здесь. Вы можете создавать новые ячейки.

,purpose_category,Кол-во клиентов,Кол-во должников,Доля невозврата
0,операции с автомобилем,4279,400,0.093480
2,получение образования,3988,369,0.092528
3,проведение свадьбы,2313,183,0.079118
1,операции с недвижимостью,10751,780,0.072551


**Вывод:** Меньшую долю невозврата составляют клиенты, которые берут кредит на "операции с недвижимостью", наибольшую же составляют клиенты, которые берут кредит на "операции с автомобилем"

### Шаг 4: общий вывод.

Рассмотрев выше, как различные показатели влияют на возврат кредита в срок, можно сделать вывод:
1. Зависимость от количества детей: Клиенты, у которых нет детей имеют наименьшую долю невозврата по сравнению с клиентами, у которых 4-ро детей. Следовательно, кредитодателю будет выгодней оформить кредит с тем клиентами, у которых показатель имеет наименьшую долю невозврата, однако здесь также нужно учитывать и другие значения, которые будут предоставлены ниже.
2. Зависимость от семейного положения: Как показали значения из таблицы, наименьшую долю невозврата имеют клиенты со статусом "вдова/вдовец", наибольшую же показали клиенты со статусом "не женат/не замужем".
3. Зависимость от уровня дохода: Клиенты из категории D(доход: от 30.001 до 50.000) имеют наименьшую долю невозврата, чем клиенты из категории С(доход: от 50.001 до 200.000).
4. Зависимоть от цели кредита: В данной категории "операции с недвижимостью" имеют наменьшую долю невозврата, чем в "операции с автомобилем".
Следовательно, можно выделить положительных и отрицательных клиентов по данным категориям:
Положительные: Вдова/вдовец, без детей, доход от 30.001 до 50.000, "операции с недвижимостью"
Отрицательные: Не женатый/не замужние, 4 ребенка, доход от 50.001 до 200.000, "операции с автомобилем"
Конечно, при данной оценке, следует также обращать свое внимание, как влияет возраст клиента на возврат кредита.